# War game analysis

http://www.lifl.fr/~jdelahay/pls/1995/030.pdf


In [235]:
from random import shuffle
import math
from lolviz import *
from itertools import *
from collections import deque
from pandas import DataFrame
from joblib import Parallel, delayed
from hashids import Hashids
hashids = Hashids()
import numpy as np


## Assumptions

- The cards are numbered from $1$ to $N$.
- A deck has 4 cards of each value. The colors are ignored so each card with the same value are equivalent.
- At the start, the deck is shuffled and divided into 2 equals parts
- At each turn, each player take the card at the top of its hand and put it on the table.
- The player with the highest value card collects both cards and returns them at the bottom of its hand. The ordered in which the cards are returned is not specified in the rules.
- If both players present the same value card, ...
- the game finishes when one player loses all its cards (he loses), or the current battle has no issue.

In [5]:
def deck(n):
    for i in range(4*n):
        yield (i // 4) + 1 

def shuffled_deck(n):
    d = list(deck(n))
    shuffle(d)
    return d

In [160]:
print (shuffled_deck(4))

for i in deck(2): print(i)

[1, 3, 2, 2, 2, 1, 4, 3, 1, 3, 3, 4, 1, 4, 2, 4]
1
1
1
1
2
2
2
2


## How many different games are possible?

A deck has $4N$ cards:

    [1a,1b,1c,1d,2a,2b,2c,2d]
    
There is $(4N)!$ ways to distribute these cards.

But in the war game, the colors are ignored: each '1' are equivalent, just like each '2' are equivalent, etc:

    [1,1,1,1,2,2,2,2]
    
We must remove the different possible combinations of each color. Each color can be distributed in 4! different ways. So we divide the first value by $(4!)^N$.


    [1,1,1,1][2,2,2,2]
    
    
    [1,1,1,2][2,2,2,1] [1,1,2,1][2,2,2,1]
    [1,1,1,2][2,2,1,2] [1,1,2,1][2,2,1,2]
    [1,1,1,2][2,1,2,2] [1,1,2,1][2,1,2,2]
    [1,1,1,2][1,2,2,2] [1,1,2,1][1,2,2,2]
    
    [1,2,1,1][2,2,2,1] [2,1,1,1][2,2,2,1]
    [1,2,1,1][2,2,1,2] [2,1,1,1][2,2,1,2]
    [1,2,1,1][2,1,2,2] [2,1,1,1][2,1,2,2]
    [1,2,1,1][1,2,2,2] [2,1,1,1][1,2,2,2]
    

    [1,1,2,2][2,2,1,1] [1,2,1,2][2,2,1,1] [1,2,2,1][2,2,1,1]
    [1,1,2,2][2,1,2,1] [1,2,1,2][2,1,2,1] [1,2,2,1][2,1,2,1]
    [1,1,2,2][2,1,1,2] [1,2,1,2][2,1,1,2] [1,2,2,1][2,1,1,2]
    [1,1,2,2][1,2,2,1] [1,2,1,2][1,2,2,1] [1,2,2,1][1,2,2,1]
    [1,1,2,2][1,2,1,2] [1,2,1,2][1,2,1,2] [1,2,2,1][1,2,1,2]
    [1,1,2,2][1,1,2,2] [1,2,1,2][1,1,2,2] [1,2,2,1][1,1,2,2]

    [2,1,1,2][2,2,1,1] [2,1,2,1][2,2,1,1] [2,2,1,1][2,2,1,1]
    [2,1,1,2][2,1,2,1] [2,1,2,1][2,1,2,1] [2,2,1,1][2,1,2,1]
    [2,1,1,2][2,1,1,2] [2,1,2,1][2,1,1,2] [2,2,1,1][2,1,1,2]
    [2,1,1,2][1,2,2,1] [2,1,2,1][1,2,2,1] [2,2,1,1][1,2,2,1]
    [2,1,1,2][1,2,1,2] [2,1,2,1][1,2,1,2] [2,2,1,1][1,2,1,2]
    [2,1,1,2][1,1,2,2] [2,1,2,1][1,1,2,2] [2,2,1,1][1,1,2,2]

    
    [1,2,2,2][2,1,1,1] [2,1,2,2][2,1,1,1]
    [1,2,2,2][1,2,1,1] [2,1,2,2][1,2,1,1]
    [1,2,2,2][1,1,2,1] [2,1,2,2][1,1,2,1]
    [1,2,2,2][1,1,1,2] [2,1,2,2][1,1,1,2]

    [2,2,1,2][2,1,1,1] [2,2,2,1][2,1,1,1]
    [2,2,1,2][1,2,1,1] [2,2,2,1][1,2,1,1]
    [2,2,1,2][1,1,2,1] [2,2,2,1][1,1,2,1]
    [2,2,1,2][1,1,1,2] [2,2,2,1][1,1,1,2]
    
    
    [2,2,2,2][1,1,1,1]
    
Here, we don't take into account which player wins or looses, so most of the previous distributions lead to the same game: either player A has the deck 1 and player B has the deck 2, or player A has the deck 2 and player B has the deck 1. The distributions that are counted twice are those for which player A and player B have different decks. So we need to enumerate the distributions where decks 1 and 2 are the same.

If players A and B have the same deck, then it must have 2 '1', 2 '2', etc. There are $\frac{(2N)!}{2^N}$ such games.

All in all, the number of possible games is half the number of distributions where deck 1 and 2 are different, plus the number of distributions where theu are the same.

$$
\begin{split}
NbPossibleGames &= \left(\frac{(4N)!}{4!^N} - \frac{(2N)!}{2^N}\right)/2 + \frac{(2N)!}{2^N} \\
                 &= \left(\frac{(4N)!}{4!^N} + \frac{(2N)!}{2^N}\right)/2
\end{split}$$

In [66]:
def nb_possible_games(n):
    return int(
        ( math.factorial(4*n) / math.pow(24,n) 
        + math.factorial(2*n) / math.pow(2,n)
        ) / 2
    )

for  n in range(1,14):
    nbgames = nb_possible_games(n)
    print(f'N={n:2} ({4*n:2} cards)  {nbgames} ({nbgames:.1E})')

N= 1 ( 4 cards)  1 (1.0E+00)
N= 2 ( 8 cards)  38 (3.8E+01)
N= 3 (12 cards)  17370 (1.7E+04)
N= 4 (16 cards)  31532760 (3.2E+07)
N= 5 (20 cards)  152770174200 (1.5E+11)
N= 6 (24 cards)  1623335272297200 (1.6E+15)
N= 7 (28 cards)  33237789624004169728 (3.3E+19)
N= 8 (32 cards)  1195230914866984691695616 (1.2E+24)
N= 9 (36 cards)  70405077040237348082159714304 (7.0E+28)
N=10 (40 cards)  6434319990707289290219959592419328 (6.4E+33)
N=11 (44 cards)  873465373058505299755221088105463283712 (8.7E+38)
N=12 (48 cards)  169958892289723968662473635773145099362369536 (1.7E+44)
N=13 (52 cards)  46012121115135518672358064931939299654060176572416 (4.6E+49)


## Enumerate all possible games

In [3]:
def split_deck(deck):
    index = int(len(deck)/2)
    # cut the deck in 2 equal parts
    g0 = tuple(deck[:index])
    g1 = tuple(deck[index:])

    # we don't take into account which player wins or looses
    return (g0,g1) if (g0<g1) else (g1,g0)

def enumerate_games(n):
    counts = [4] * n
    game = [None]*(4*n)
    
    def recurse(index, hashes):
         for value in range(n): # each card can have a value up to n
            
            if counts[value] == 0: # no more card with this value
                continue
            
            counts[value] -= 1
            
            game[index] = value+1
            
            if index == 4*n - 1:
                g = split_deck(game)
                
                # check that this distribution was not already returned
                h = hash(g)
                if not h in hashes:
                    hashes.add(h)
                    yield g
                
            else:
                yield from recurse(index+1, hashes)
            
            counts[value] += 1
       
    yield from recurse(0, set())

    
num = 0
for game in enumerate_games(2):
    num += 1
    print(f'{num:5}    {game}')


    1    ((1, 1, 1, 1), (2, 2, 2, 2))
    2    ((1, 1, 1, 2), (1, 2, 2, 2))
    3    ((1, 1, 1, 2), (2, 1, 2, 2))
    4    ((1, 1, 1, 2), (2, 2, 1, 2))
    5    ((1, 1, 1, 2), (2, 2, 2, 1))
    6    ((1, 1, 2, 1), (1, 2, 2, 2))
    7    ((1, 1, 2, 1), (2, 1, 2, 2))
    8    ((1, 1, 2, 1), (2, 2, 1, 2))
    9    ((1, 1, 2, 1), (2, 2, 2, 1))
   10    ((1, 1, 2, 2), (1, 1, 2, 2))
   11    ((1, 1, 2, 2), (1, 2, 1, 2))
   12    ((1, 1, 2, 2), (1, 2, 2, 1))
   13    ((1, 1, 2, 2), (2, 1, 1, 2))
   14    ((1, 1, 2, 2), (2, 1, 2, 1))
   15    ((1, 1, 2, 2), (2, 2, 1, 1))
   16    ((1, 2, 1, 1), (1, 2, 2, 2))
   17    ((1, 2, 1, 1), (2, 1, 2, 2))
   18    ((1, 2, 1, 1), (2, 2, 1, 2))
   19    ((1, 2, 1, 1), (2, 2, 2, 1))
   20    ((1, 2, 1, 2), (1, 2, 1, 2))
   21    ((1, 2, 1, 2), (1, 2, 2, 1))
   22    ((1, 2, 1, 2), (2, 1, 1, 2))
   23    ((1, 2, 1, 2), (2, 1, 2, 1))
   24    ((1, 2, 1, 2), (2, 2, 1, 1))
   25    ((1, 2, 2, 1), (1, 2, 2, 1))
   26    ((1, 2, 2, 1), (2, 1, 1, 2))
   27    ((1

## Play a game



In [31]:
def play_game(packs):
    playerA = deque(packs[0])
    playerB = deque(packs[1])
    stack = deque()
    tricks = 0
    cards = 0
    hashes = []
    
    
    
    #print(packs)
    
    while (len(playerA) and len(playerB)):
        
        if not stack:
            h = hash(tuple(list(playerA) + [0] + list(playerB)))
            if h in hashes:
                #print(hashes, h)
                offset = hashes.index(h)
                period = len(hashes) - offset
                return packs, 3, offset, period

            hashes += [h]
        
        a = playerA.popleft()
        b = playerB.popleft()
        #print(f'  a:{a}, b:{b}')

        cards += 1
        if not stack: tricks += 1    

        if (a > b):
            playerA.extend([a,b])
            playerA.extend(stack)
            stack.clear()
            #print(f'A:{playerA}, B:{playerB}')
        elif (a < b):
            playerB.extend([b,a])
            playerB.extend(stack)
            stack.clear()
            #print(f'A:{playerA}, B:{playerB}')
        else:
            stack.extendleft([a,b])
            #print(f'    stack:{stack}')
    
    winner = 1 if len(playerA) \
        else 2 if len(playerB) \
        else 0
    
    return packs, winner, tricks, cards
    
winner_name = ['None','A','B','Infinite']
game, winner, tricks, cards = play_game( split_deck(shuffled_deck(3)) )
print(game, winner_name[winner], tricks, cards)

# winner, tricks, cards = play_game( ([1,1,2,2],[1,1,2,2]) )
# print(winner_name[winner], tricks, cards)
# winner, tricks, cards = play_game( ([1,1,2,1],[1,2,2,2]) )
# print(winner_name[winner], tricks, cards)
# winner, tricks, cards = play_game( ([1,2,1,2],[2,1,1,2]) )
# print(winner_name[winner], tricks, cards)
# winner, tricks, cards = play_game( ([1,1,1,2],[2,2,1,2]) )
# print(winner_name[winner], tricks, cards)
# winner, tricks, cards = play_game( ([2,1,1,2],[2,2,1,1]) )
# print(winner_name[winner], tricks, cards)
# winner, tricks, cards = play_game( ([2,1,1,2],[1,2,2,1]) )
# print(winner_name[winner], tricks, cards)

# winner, tricks, cards = play_game( ((3,2,1,1,2,1), (2,1,3,2,3,3)) )
# print(winner_name[winner], tricks, cards)


((1, 3, 3, 2, 1, 1), (2, 2, 3, 3, 1, 2)) B 8 12


In [10]:
all_games = []
for g in enumerate_games(2):
    game, winner, tricks, cards = play_game(g)
    all_games += [{'game':game, 'winner':winner, 'tricks':tricks, 'cards':cards}]

df = DataFrame(all_games, columns=['game', 'winner', 'tricks', 'cards'])
#print(df)
#print(df.info())
print(df.describe())

          winner     tricks      cards
count  38.000000  38.000000  38.000000
mean    1.263158   2.973684   5.263158
std     0.890921   1.283720   1.427230
min     0.000000   1.000000   4.000000
25%     0.000000   2.000000   4.000000
50%     2.000000   3.000000   5.000000
75%     2.000000   3.750000   6.000000
max     2.000000   5.000000   8.000000


In [309]:
from numba import jit

def fnv64(data):

    hash_ = 0xcbf29ce484222325

    for b in data:

        hash_ *= 0x100000001b3

        hash_ &= 0xffffffffffffffff

        hash_ ^= b

    return hash_


def myhash(data):
    base_size = np.uint64(len(data) / 8)
    base = base_size ** np.arange(len(data))
    print (base)
    hashed_array = (base * data)
    print(hashed_array)
    return hashed_array.sum()
    

#@jit(nopython=True)
def find_first(item, vec, max):
    """return the index of the first occurence of item in vec"""
    for i in range(max):
        if item == vec[i]:
            return i
    return -1

#@jit
def play_game_2(deck):
    
    L = int(len(deck))
    L2 = int(L / 2)
    
    topA = 0
    endA = L2
    topB = L
    endB = L+L2
    
    #cards = [0] * (L*2)
    cards = np.zeros(2*L)
    cards[topA:endA] = deck[:L2]
    cards[topB:endB] = deck[L2:]
    
    stack = np.zeros(L2)
    endStack = 0
    
    #print("deck ",deck)
    #print(f'      a:_, b:_, cards:{cards}, stack:{stack}')

    ntricks = 0
    ncards = 0
    
    hashes = np.zeros(5000)
    endHash = 0

    while (endA>topA and endB>topB):
        
        if endStack == 0:
            #h = hash(tuple(list(playerA) + [0] + list(playerB)))
            h = myhash(cards)
            #h = hash(cards.tostring())
            offset = find_first(h,hashes,endHash)
            if offset >= 0:
                #print(hashes, h)
                period = endHash - offset
                return deck, 3, offset, period

            hashes[endHash] = h
            endHash += 1
        
        a = cards[topA]
        b = cards[topB]
        #cards[:-1] = cards[1:]
        np.roll(cards,-1)
#--#
        cards[L-1] = cards[2*L-1] = 0
#--#
        endA -= 1
        endB -= 1

        ncards += 1
        if endStack == 0: ntricks += 1    

        #print(f'{ncards:2}    a:{a}, b:{b}, cards:{cards}, stack:{stack}')

        if (a > b):
            #playerA.extend([a,b])
            cards[endA] = a; endA += 1
            cards[endA] = b; endA += 1
            #playerA.extend(stack)
            for i in range(endStack):
                cards[endA] = stack[endStack-1-i]; endA += 1
                cards[endA] = stack[endStack-1-i]; endA += 1
            #stack.clear()
#--#
            for i in range(endStack): stack[i] = 0
#--#
            endStack = 0
            #print(f'                cards:{cards}')
        
        elif (a < b):
            #playerB.extend([b,a])
            cards[endB] = b; endB += 1
            cards[endB] = a; endB += 1
            #playerB.extend(stack)
            for i in range(endStack):
                cards[endB] = stack[endStack-1-i]; endB += 1
                cards[endB] = stack[endStack-1-i]; endB += 1
            #stack.clear()
#--#
            for i in range(endStack): stack[i] = 0
#--#
            endStack = 0
            #print(f'                cards:{cards}')
        
        else:
            #stack.extendleft([a,b])
            stack[endStack] = a; endStack += 1
            #print(f'                stack:{stack}')
    
    winner = 1 if endA>topA \
        else 2 if endB>topB \
        else 0
    
    return deck, winner, ntricks, ncards

    
#play_game_2(shuffled_deck(3))
play_game_2([3,2,1,1,2,1, 2,1,3,2,3,3])

[         1          3          9         27         81        243
        729       2187       6561      19683      59049     177147
     531441    1594323    4782969   14348907   43046721  129140163
  387420489 1162261467 -808182895 1870418611 1316288537 -346101685]
[ 3.00000000e+00  6.00000000e+00  9.00000000e+00  2.70000000e+01
  1.62000000e+02  2.43000000e+02  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.06288200e+06  1.59432300e+06  1.43489070e+07  2.86978140e+07
  1.29140163e+08  3.87420489e+08  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00]
[         1          3          9         27         81        243
        729       2187       6561      19683      59049     177147
     531441    1594323    4782969   14348907   43046721  129140163
  387420489 1162261467 -808182895 1870418611 1316288537 -346101685]
[ 3.00000000e+00  6.00000000e+00  9.00000000e+00  2.70000000e+01
  1.62

([3, 2, 1, 1, 2, 1, 2, 1, 3, 2, 3, 3], 1, 6, 6)

In [299]:
%timeit all_games = [play_game(game) for game in enumerate_games(2)]

game_list = [[c for a in game for c in a] for game in enumerate_games(2)]
#print(game_list)
%timeit all_games = [play_game_2(game) for game in game_list]

535 µs ± 1.34 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.71 ms ± 6.87 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [260]:
import cProfile

game_list = [[c for a in game for c in a] for game in enumerate_games(3)]
cProfile.run('for game in game_list: play_game_2(game)',sort='cumtime')

         698655 function calls in 1.510 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.510    1.510 {built-in method builtins.exec}
        1    0.200    0.200    1.510    1.510 <string>:1(<module>)
    17370    0.657    0.000    1.310    0.000 <ipython-input-257-f30651ed4d0f>:20(play_game_2)
   165888    0.235    0.000    0.650    0.000 <ipython-input-257-f30651ed4d0f>:15(myhash)
   165888    0.377    0.000    0.377    0.000 {built-in method numpy.core.multiarray.array}
   165888    0.022    0.000    0.022    0.000 {built-in method builtins.hash}
   165888    0.016    0.000    0.016    0.000 {method 'tobytes' of 'memoryview' objects}
    17370    0.002    0.000    0.002    0.000 {built-in method builtins.len}
      360    0.000    0.000    0.000    0.000 {method 'index' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [267]:
cProfile.run('for data in game_list: fnv64(data)',sort='cumtime')

         17373 function calls in 0.040 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.040    0.040 {built-in method builtins.exec}
        1    0.004    0.004    0.040    0.040 <string>:1(<module>)
    17370    0.037    0.000    0.037    0.000 <ipython-input-263-75e5b3d3902c>:1(fnv64)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [270]:
np_game_list = [ np.array([c for a in game for c in a], dtype=np.uint8) for game in enumerate_games(3) ]
cProfile.run('for data in np_game_list: myhash(data)',sort='cumtime')

         52113 function calls in 0.021 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.021    0.021 {built-in method builtins.exec}
        1    0.003    0.003    0.021    0.021 <string>:1(<module>)
    17370    0.014    0.000    0.018    0.000 <ipython-input-268-5834c52f0a90>:15(myhash)
    17370    0.002    0.000    0.002    0.000 {built-in method builtins.hash}
    17370    0.002    0.000    0.002    0.000 {method 'tobytes' of 'memoryview' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [283]:
x = np.arange(10)
np.roll(x,-2)

array([2, 3, 4, 5, 6, 7, 8, 9, 0, 1])